# ANALYSE DES DONNÉES

In [2]:
import pickle
import math
import numpy as np
import os
import sklearn
from sklearn.decomposition import PCA
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler



with open("/Users/amelle/Desktop/Sogeti/gen-one/fd_data_gen_one_s11.pickle", "rb") as f:
    data_s11 = pickle.load(f)

with open("/Users/amelle/Desktop/Sogeti/gen-one/metadata_gen_one.pickle", "rb") as f:
    metadata = pickle.load(f)




with open("/Users/amelle/Desktop/Sogeti/datasets/gen-one/clean/fd_data_gen_one_s11.pickle", "rb") as f:
    fd_data = pickle.load(f)

with open("/Users/amelle/Desktop/Sogeti/datasets/gen-one/clean/metadata_gen_one.pickle", "rb") as f:
    metadata_data = pickle.load(f)
    
    

with open("/Users/amelle/Desktop/Sogeti/datasets/gen-one/clean/train_data.pickle", "rb") as f:
    train_data = pickle.load(f)

with open("/Users/amelle/Desktop/Sogeti/datasets/gen-one/clean/train_labels.pickle", "rb") as f:
    train_labels = pickle.load(f)

with open("/Users/amelle/Desktop/Sogeti/datasets/gen-one/clean/train_md.pickle", "rb") as f:
    train_md = pickle.load(f)
    


with open("/Users/amelle/Desktop/Sogeti/datasets/gen-one/clean/test_data.pickle", "rb") as f:
    test_data = pickle.load(f)

with open("/Users/amelle/Desktop/Sogeti/datasets/gen-one/clean/test_labels.pickle", "rb") as f:
    test_labels = pickle.load(f)

with open("/Users/amelle/Desktop/Sogeti/datasets/gen-one/clean/test_md.pickle", "rb") as f:
    test_md = pickle.load(f)


In [3]:
print (test_labels.shape)
print (train_labels.shape)

(50,)
(199,)


<style>
    div.output_scroll {
        height: 400px;  /* Définit une hauteur fixe avec une barre de défilement */
        overflow-y: scroll;
    }
</style>

#### Chargement du dataset principal et exploration des données et des métadonnées

In [6]:
print(fd_data)

[[[ 0.77276859+0.60698369j  0.77286743+0.60692611j
    0.77286839+0.60683238j ...  0.77233386+0.60739237j
    0.77253262+0.60728794j  0.77273545+0.60718263j]
  [ 0.78432414+0.59168948j  0.7844515 +0.59163127j
    0.78442235+0.59157939j ...  0.78395586+0.5921893j
    0.7841345 +0.59207824j  0.7843103 +0.59192498j]
  [ 0.79525368+0.57617232j  0.79535535+0.57612365j
    0.79533053+0.57604345j ...  0.79492659+0.57677655j
    0.79509979+0.57659964j  0.79522991+0.57644758j]
  ...
  [-0.08254912+0.24635747j -0.08540133+0.24947416j
   -0.08362449+0.24960577j ... -0.08371444+0.25233713j
   -0.08132568+0.24992455j -0.08178679+0.24718514j]
  [-0.07975056+0.24808725j -0.08233818+0.25152835j
   -0.08042578+0.2514465j  ... -0.08030329+0.25421239j
   -0.07813598+0.25170557j -0.07872076+0.24913079j]
  [-0.07676874+0.24976993j -0.07904621+0.25345226j
   -0.07714552+0.2532364j  ... -0.07687031+0.25602756j
   -0.07485886+0.25337649j -0.07545079+0.25093989j]]

 [[ 0.77298746+0.60728702j  0.7729675 +0.6071

In [7]:
print(fd_data.shape)

(323, 1001, 72)


In [8]:
for i, entry in enumerate(metadata_data[:8]):  
    print(f"--- Metadata {i} ---")
    for key, value in entry.items():
        print(f"{key}: {value}")
    print()

--- Metadata 0 ---
n_expt: 1
id: 1
phant_id: A2F1
tum_rad: 3.0
tum_shape: sphere
tum_x: 3.0
tum_y: -3.0
tum_z: nan
birads: 1
adi_ref_id: 6
emp_ref_id: 12
date: 2018-02-28
n_session: 1
ant_rad: 21.0
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: 5
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 1 ---
n_expt: 2
id: 2
phant_id: A2F1
tum_rad: 2.0
tum_shape: sphere
tum_x: 3.0
tum_y: -3.0
tum_z: nan
birads: 1
adi_ref_id: 6
emp_ref_id: 12
date: 2018-02-28
n_session: 1
ant_rad: 21.0
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: 5
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 2 ---
n_expt: 3
id: 3
phant_id: A2F1
tum_rad: 2.0
tum_shape: spiculated
tum_x: 3.0
tum_y: -3.0
tum_z: nan
birads: 1
adi_ref_id: 6
emp_ref_id: 12
date: 2018-02-28
n_session: 1
ant_rad: 21.0
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: 5
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 3 ---
n_expt: 4
id: 4
phant_id: A2F1
tum_rad: 1.0
tum_shape: sphere
tum_x: 3.0
tum_y: -3.0
tum_z: nan

In [9]:
for i, entry in enumerate(train_md[:8]):  
    print(f"--- Metadata {i} ---")
    for key, value in entry.items():
        print(f"{key}: {value}")
    print()

--- Metadata 0 ---
n_expt: 3
id: 130
phant_id: A1F4
tum_rad: 2.0
tum_shape: spiculated
tum_x: 1.5
tum_y: -3.0
tum_z: nan
birads: 4
adi_ref_id: 133
emp_ref_id: 134
date: 2018-03-04
n_session: 10
ant_rad: 20.0
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: 132
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 1 ---
n_expt: 9
id: 71
phant_id: A3F3
tum_rad: 2.0
tum_shape: sphere
tum_x: -2.25
tum_y: 0.75
tum_z: nan
birads: 2
adi_ref_id: 74
emp_ref_id: 69
date: 2019-06-12
n_session: 6
ant_rad: 21.9
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: nan
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 2 ---
n_expt: 1
id: 276
phant_id: A2F5
tum_rad: 1.0
tum_shape: sphere
tum_x: 0.75
tum_y: -4.0
tum_z: nan
birads: 4
adi_ref_id: 282
emp_ref_id: 283
date: 2019-03-19
n_session: 26
ant_rad: 21.5
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: nan
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 3 ---
n_expt: 19
id: 275
phant_id: A3F2
tum_rad: 3.0
tum_shape: sphere
tum_x: 

In [10]:
for i, entry in enumerate(train_md[:3]):  
    print(f"--- Metadata {i} ---")
    for key, value in entry.items():
        print(f"{key}: {value}")
    print()

--- Metadata 0 ---
n_expt: 3
id: 130
phant_id: A1F4
tum_rad: 2.0
tum_shape: spiculated
tum_x: 1.5
tum_y: -3.0
tum_z: nan
birads: 4
adi_ref_id: 133
emp_ref_id: 134
date: 2018-03-04
n_session: 10
ant_rad: 20.0
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: 132
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 1 ---
n_expt: 9
id: 71
phant_id: A3F3
tum_rad: 2.0
tum_shape: sphere
tum_x: -2.25
tum_y: 0.75
tum_z: nan
birads: 2
adi_ref_id: 74
emp_ref_id: 69
date: 2019-06-12
n_session: 6
ant_rad: 21.9
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: nan
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 2 ---
n_expt: 1
id: 276
phant_id: A2F5
tum_rad: 1.0
tum_shape: sphere
tum_x: 0.75
tum_y: -4.0
tum_z: nan
birads: 4
adi_ref_id: 282
emp_ref_id: 283
date: 2019-03-19
n_session: 26
ant_rad: 21.5
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: nan
fib_x: nan
fib_y: nan
tum_in_fib: 



In [11]:
for i, entry in enumerate(test_md[:8]):  
    print(f"--- Metadata {i} ---")
    for key, value in entry.items():
        print(f"{key}: {value}")
    print()

--- Metadata 0 ---
n_expt: 7
id: 110
phant_id: A1F1
tum_rad: 3.0
tum_shape: sphere
tum_x: nan
tum_y: nan
tum_z: nan
birads: 2
adi_ref_id: 114
emp_ref_id: 115
date: 2018-03-02
n_session: 8
ant_rad: 20.0
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: nan
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 1 ---
n_expt: 9
id: 143
phant_id: A3F1
tum_rad: 2.0
tum_shape: spiculated
tum_x: -0.5
tum_y: 1.25
tum_z: nan
birads: 1
adi_ref_id: 145
emp_ref_id: 146
date: 2018-03-13
n_session: 11
ant_rad: 21.0
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: nan
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 2 ---
n_expt: 1
id: 104
phant_id: A1F1
tum_rad: 3.0
tum_shape: sphere
tum_x: 2.5
tum_y: -3.0
tum_z: nan
birads: 2
adi_ref_id: 109
emp_ref_id: 115
date: 2018-03-02
n_session: 8
ant_rad: 20.0
ant_z: nan
fib_ang: nan
adi_x: nan
adi_y: nan
fib_ref_id: 108
fib_x: nan
fib_y: nan
tum_in_fib: 

--- Metadata 3 ---
n_expt: 10
id: 156
phant_id: A3F3
tum_rad: 1.0
tum_shape: sphere
tum_x: -

In [12]:
visualisation_labels = []
for entry in metadata_data:
    rad = entry.get('tum_rad', float('nan'))
    if math.isnan(rad):
        visualisation_labels.append(0)  # pas de tumeur
    else:
        visualisation_labels.append(1)  # tumeur

print("Labels :", visualisation_labels[:323]) 

Labels : [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0]


In [13]:
count_tum = 0    # tum_rad > 0
count_no_tum = 0 # tum_rad = 0
count_nan = 0    # tum_rad = NaN

for entry in metadata:
    rad = entry.get('tum_rad', float('nan'))
    
    if math.isnan(rad):
        count_nan += 1
    elif rad > 0:
        count_tum += 1
    else:
        count_no_tum += 1

print("Nombre de scans avec tum_rad > 0  :", count_tum)
print("Nombre de scans avec tum_rad = 0  :", count_no_tum)
print("Nombre de scans avec tum_rad = NaN:", count_nan)

Nombre de scans avec tum_rad > 0  : 124
Nombre de scans avec tum_rad = 0  : 0
Nombre de scans avec tum_rad = NaN: 199


# DÉBUT DU MACHINE LEARNING

## Régression Logistique

In [16]:
import logging
import sys

def get_notebook_logger(name="notebook_logger", level=logging.DEBUG):
    """
    Initialise un logger compatible avec Jupyter Notebook.
    
    Parameters
    ----------
    name : str
        Nom du logger.
    level : int
        Niveau de logging (DEBUG, INFO, WARNING, ERROR, CRITICAL).
    
    Returns
    -------
    logger : logging.Logger
        Logger configuré.
    """
    
    logger = logging.getLogger(name)

    if logger.hasHandlers():
        logger.handlers.clear()

    logger.setLevel(level)

    log_format = logging.Formatter("[%(asctime)s][%(levelname)s] %(message)s")

    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setFormatter(log_format)

    logger.addHandler(stdout_handler)

    return logger

logger = get_notebook_logger()
logger.info("Logger pour Jupyter Notebook initialisé avec succès.")


[2025-03-15 17:37:01,564][INFO] Logger pour Jupyter Notebook initialisé avec succès.


In [17]:
def normalize_samples(data):
    """Normalizes each sample in data to have unity maximum

    Parameters
    ----------
    data : array_like
        3D array of the features for each sample (assumes 2D features)

    Returns
    -------
    normalized_data : array_like
        Array of the features for each sample, normalized so that the
        max value is unity for each sample
    """

    # Assert that data must be 3D
    assert len(np.shape(data)) == 3, 'Error: data must have 3 dim'

    normalized_data = np.ones_like(data)  # Init array to return

    # For each sample
    for sample_idx in range(np.size(data, axis=0)):

        # Normalize to have max of unity
        normalized_data[sample_idx, :, :] = (data[sample_idx, :, :] /
                                             np.max(data[sample_idx, :, :]))

    return normalized_data


In [18]:
"""
Tyson Reimer
University of Manitoba
September 14th, 2019
"""

###############################################################################
__LEARN_RATE = 1    # Set the learning rate for gradient descent
__MAX_ITER = 10000  # Set the number of iterations used to train

##############################################################################


def get_best_acc(labels, preds, fixed_threshold=-1.0):
    """Finds the best threshold, acc, sens and spec for a classifier

    Parameters
    ----------
    labels : list, array_like
        The binary class labels of each sample
    preds : list, array_like
        The predicted scores of each sample
    fixed_threshold : float
        If set to -1.0, will not use a fixed threshold. If set to any
        other value, will find the acc/sens/spec at that threshold

    Returns
    -------
    best_threshold : float
        The threshold that results in the highest diagnostic accuracy,
        if fixed_threshold == -1.0, otherwise is fixed_threshold
    best_acc : float
        The best diagnostic accuracy
    best_sens : float
        The best sensitivity
    best_spec : float
        The best specificity
    """

    labels = np.array(labels)  # Convert to np array
    preds = np.array(preds)  # Convert to np array

    # Init the best acc/sens/spec/threshold
    best_acc = -1
    best_sens = -1
    best_spec = -1
    best_threshold = 0

    if fixed_threshold == -1.0:  # If fixed_threshold at default value

        # Define var for possible thresholds
        possible_thresholds = np.linspace(0, 1, 1000)

    else:  # If fixed_threshold specified to be not-default
        assert 0 <= fixed_threshold <= 1, \
                "Error: fixed_threshold must be between 0 and 1"

        # Set possible thresholds to be list of only this threshold
        possible_thresholds = [fixed_threshold]

    for thresh in possible_thresholds:  # For each threshold

        # Find the true positives/negatives, false postives/negatives
        tp = np.sum(np.logical_and(labels == 1, preds > thresh))
        tn = np.sum(np.logical_and(labels == 0, preds < thresh))
        fn = np.sum(np.logical_and(labels == 1, preds < thresh))
        fp = np.sum(np.logical_and(labels == 0, preds > thresh))

        # Get the accuracy/sensitivity/specificity at this threshold
        acc = (tp + tn) / (fn + fp + tn + tp)
        sens = tp / (tp + fn) if (tp + fn) != 0 else 0
        spec = tn / (tn + fp) if (tn + fp) != 0 else 0

        if acc > best_acc:  # If this is the best accuracy thus far

            # Store the acc/sens/spec/threshold as the *best*
            best_acc = acc
            best_threshold = thresh
            best_sens = sens
            best_spec = spec

    logger.info('\tbest threshold:\t%.3f' % best_threshold)

    return best_threshold, best_acc, best_sens, best_spec


def report_results(data, labels, return_threshold=False, threshold=-1.0):
    """Reports the classification results to the logger

    Parameters
    ----------
    data : array_like
        The features for each sample in the data
    labels : list, array_like
        The binary class labels for each sample in the data
    return_threshold : bool
        If True, will also return the threshold used to compute the
        classification results
    threshold : float
        If -1.0, will find the best threshold and use that to find
        classification results. Otherwise, will use its value to find
        classification results.

    Returns
    -------
    threshold : float
        The threshold - only returned if return_threshold
    acc : float
        The accuracy of the classification predictions
    roc_score : float
        The ROC AUC score of the classification predictions
    sens : float
        The sensitivity of the classification predictions
    spec : float
        The specificity of the classification predictions
    """

    # Get the predicted scores for each sample
    pred_probs = logreg.predict_proba(data)[:, 1]

    # Find the threshold, acc/sens/spec for these predictions
    threshold, acc, sens, spec = get_best_acc(labels, pred_probs,
                                              fixed_threshold=threshold)

    # Compute the ROC AUC score for the predictions
    roc_score = roc_auc_score(labels, pred_probs)

    # Report to the logger
    logger.info('\t\tAcc:\t%.3f' % acc)
    logger.info('\t\tROC:\t%.3f' % roc_score)
    logger.info('\t\tSens:\t%.3f' % sens)
    logger.info('\t\tSpec:\t%.3f' % spec)

    if return_threshold:  # If returning the threshold, then do that
        return threshold, acc, roc_score, sens, spec

    else:  # If not returning the threshold, then return metrics only
        return acc, roc_score, sens, spec

###############################################################################
# Paramètres pour la recherche de la fenêtre optimale
window_size = 10  
num_windows = 100  
n_repeats = 10     

window_starts = np.arange(0, 1000 - window_size + 1, step=window_size)
best_avg_acc = 0
best_window = None

# Parcourir chaque fenêtre pour sélectionner la meilleure
for start_idx in window_starts:
    end_idx = start_idx + window_size
    freq_indices = np.arange(start_idx, end_idx)  # 10 fréquences consécutives

    acc_list = []
    for rep in range(n_repeats):
        train_data_fd = np.imag(train_data[:, freq_indices, :])
        test_data_fd = np.imag(test_data[:, freq_indices, :])
    
        train_data_norm = normalize_samples(train_data_fd)
        test_data_norm = normalize_samples(test_data_fd)
    
        train_data_flat = np.reshape(train_data_norm, [np.size(train_data_norm, axis=0), window_size * 72])
        test_data_flat = np.reshape(test_data_norm, [np.size(test_data_norm, axis=0), window_size * 72])
    
        data_all = np.concatenate((train_data_flat, test_data_flat), axis=0)
        labels_all = np.concatenate((train_labels, test_labels), axis=0)
    
        X_train, X_test, y_train, y_test = train_test_split(
            data_all, labels_all, test_size=0.2, stratify=labels_all, random_state=42 + rep
        )

        

        # Après avoir effectué le train/test split :
        scaler = RobustScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    

        logreg = LogisticRegression(solver='liblinear', penalty='l2', max_iter=__MAX_ITER)

        # Fit the classifier
        logreg.fit(X_train, y_train)
    
        # On récupère uniquement l'accuracy pour la sélection
        _, acc, _, _ = report_results(X_test, y_test)
        acc_list.append(acc)
    
    avg_acc = np.mean(acc_list)
    print(f"Fenêtre {start_idx}-{end_idx} : Accuracy moyenne = {avg_acc:.3f}")
    
    if avg_acc > best_avg_acc:
        best_avg_acc = avg_acc
        best_window = (start_idx, end_idx)

print(f"\nMeilleure fenêtre trouvée : {best_window} avec une Accuracy moyenne de {best_avg_acc:.3f}")


# Utiliser la meilleure fenêtre trouvée pour le reste du processus
start_idx, end_idx = best_window
freq_indices = np.arange(start_idx, end_idx)

# Extraire la partie imaginaire des données sur la meilleure fenêtre
train_data_fd = np.imag(train_data[:, freq_indices, :])
test_data_fd = np.imag(test_data[:, freq_indices, :])

train_data = normalize_samples(train_data_fd)
test_data = normalize_samples(test_data_fd)

train_data = np.reshape(train_data, [np.size(train_data, axis=0), window_size * 72])
test_data = np.reshape(test_data, [np.size(test_data, axis=0), window_size * 72])


###############################################################################

if __name__ == '__main__':

    # Define the logger and output init statements
    logger.info('\tBEGINNING...')
    logger.info('\tMax Iter: %d\tLearn Rate: %.3f' % (__MAX_ITER,
                                                      __LEARN_RATE))

    # The number of runs over which results will be averaged
    n_runs = 100

    # Init lists for storing the metrics on the train/test sets at
    # each iteration
    train_acc, train_roc = [], []
    train_sens, train_spec = [], []
    test_acc, test_roc = [], []
    test_sens, test_spec = [], []

   # Combiner les données d'entraînement et de test en un seul ensemble 
    data_all = np.concatenate((train_data, test_data), axis=0)
    labels_all = np.concatenate((train_labels, test_labels), axis=0)

    for run_idx in range(n_runs):  # Pour chaque run
        train_data_split, test_data_split, train_labels_split, test_labels_split = train_test_split(
        data_all, labels_all, test_size=0.2, stratify=labels_all
        )

        scaler = StandardScaler()
        train_data = scaler.fit_transform(train_data_split)
        test_data = scaler.transform(test_data_split)
        train_labels = train_labels_split
        test_labels= test_labels_split

        logger.info('Working on run [%3d / %3d]' % (run_idx + 1, n_runs))
        logger.info('\tTest Set NumPos: %d\tNumNeg: %d' % (np.sum(test_labels), np.sum(1 - test_labels)))

        
        # Utiliser la LogisticRegression de scikit-learn
        logreg = LogisticRegression(solver='liblinear', penalty='l2', max_iter=__MAX_ITER)
        logreg.fit(train_data, train_labels)

        # Get the results and threshold from the training set
        logger.info('\tTrain Data:')
        threshold, acc, roc, sens, spec = report_results(train_data,
                                                         train_labels,
                                                         return_threshold=True,
                                                         threshold=-1.0)

        # Store the metrics for the training set
        train_acc.append(acc)
        train_roc.append(roc)
        train_sens.append(sens)
        train_spec.append(spec)

        # Get the results on the test set using threshold from the
        # training set
        logger.info('\tTest Data:')
        acc, roc, sens, spec = report_results(test_data, test_labels,
                                              threshold=threshold)

        # Store the metrics for the test set
        test_acc.append(acc)
        test_roc.append(roc)
        test_sens.append(sens)
        test_spec.append(spec)

    # Convert the lists to np arrays
    train_acc = np.array(train_acc)
    train_roc = np.array(train_roc)
    test_acc = np.array(test_acc)
    test_roc = np.array(test_roc)
    test_sens = np.array(test_sens)
    test_spec = np.array(test_spec)

    # Report final results to logger
    logger.info('Train Final:')
    logger.info('Acc:\t%.3f +/- %.3f' %
                (np.mean(train_acc), np.std(train_acc)))
    logger.info('ROC:\t%.3f +/- %.3f' %
                (np.mean(train_roc), np.std(train_roc)))

    logger.info('Test  Final:')
    logger.info('Acc:\t%.6f +/- %.6f' %
                (np.mean(test_acc), np.std(test_acc)))
    logger.info('ROC:\t%.6f +/- %.6f' %
                (np.mean(test_roc), np.std(test_roc)))
    logger.info('Sens:\t%.6f +/- %.6f' %
                (np.mean(test_sens), np.std(test_sens)))
    logger.info('Spec:\t%.6f +/- %.6f' %
                (np.mean(test_spec), np.std(test_spec)))


[2025-03-15 17:37:01,817][INFO] 	best threshold:	0.234
[2025-03-15 17:37:01,820][INFO] 		Acc:	0.780
[2025-03-15 17:37:01,821][INFO] 		ROC:	0.781
[2025-03-15 17:37:01,821][INFO] 		Sens:	0.840
[2025-03-15 17:37:01,822][INFO] 		Spec:	0.720
[2025-03-15 17:37:02,067][INFO] 	best threshold:	0.246
[2025-03-15 17:37:02,068][INFO] 		Acc:	0.760
[2025-03-15 17:37:02,069][INFO] 		ROC:	0.749
[2025-03-15 17:37:02,070][INFO] 		Sens:	0.920
[2025-03-15 17:37:02,070][INFO] 		Spec:	0.600
[2025-03-15 17:37:02,339][INFO] 	best threshold:	0.613
[2025-03-15 17:37:02,341][INFO] 		Acc:	0.820
[2025-03-15 17:37:02,342][INFO] 		ROC:	0.790
[2025-03-15 17:37:02,344][INFO] 		Sens:	0.920
[2025-03-15 17:37:02,345][INFO] 		Spec:	0.720
[2025-03-15 17:37:02,631][INFO] 	best threshold:	0.607
[2025-03-15 17:37:02,633][INFO] 		Acc:	0.940
[2025-03-15 17:37:02,634][INFO] 		ROC:	0.955
[2025-03-15 17:37:02,634][INFO] 		Sens:	0.960
[2025-03-15 17:37:02,635][INFO] 		Spec:	0.920
[2025-03-15 17:37:02,863][INFO] 	best threshold:	0.3

In [19]:
# Définition de la plage de fréquences 
freq_start = 1.0  # GHz (fréquence minimale)
freq_end = 8.0    # GHz (fréquence maximale)
num_points = 1001  

frequencies = np.linspace(freq_start, freq_end, num_points)

freq_min = 1.56  # GHz
freq_max = 1.625  # GHz 1.616

# Trouver les indices correspondant à cette fenêtre
freq_indices = np.where((frequencies >= freq_min) & (frequencies <= freq_max))[0]
print(freq_indices)


[80 81 82 83 84 85 86 87 88 89]
